In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from text_untils import *
import json

In [2]:
def getQuestions(database, collection):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database]
    cl = db[collection]

    pipeline = [
        {
            "$match": {
                "title": {"$nin": [None, ""]},
                "quote.content": {"$exists": True, "$not": {"$size": 0}}
            }
        }
    ]
    docs = list(cl.aggregate(pipeline))
    # client.close()
    return docs

In [3]:
documents = getQuestions('lawlaboratory', 'questions')

In [4]:
df = pd.DataFrame(documents)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63817 entries, 0 to 63816
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          63817 non-null  object
 1   title        63817 non-null  object
 2   date_answer  63817 non-null  object
 3   field        63817 non-null  object
 4   source_url   63817 non-null  object
 5   reference    63817 non-null  object
 6   quote        63817 non-null  object
 7   conclusion   63817 non-null  object
dtypes: object(8)
memory usage: 3.9+ MB


In [5]:
keywords = ["Thông tư", "Quyết định", "Quy định", "Nghị định"]

def contains_keywords(text, keywords):
    return any(keyword in text for keyword in keywords)

In [6]:
df = df[~df['reference'].apply(lambda x: contains_keywords(str(x), keywords))]

In [7]:
df = df[df['reference'].str.contains('Luật', case=False, na=False)]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24647 entries, 0 to 63814
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          24647 non-null  object
 1   title        24647 non-null  object
 2   date_answer  24647 non-null  object
 3   field        24647 non-null  object
 4   source_url   24647 non-null  object
 5   reference    24647 non-null  object
 6   quote        24647 non-null  object
 7   conclusion   24647 non-null  object
dtypes: object(8)
memory usage: 1.7+ MB


In [9]:
df.drop(columns=['quote', 'conclusion', 'date_answer', 'field', 'source_url'], inplace=True)

In [10]:
df.reset_index(drop=True, inplace=True)
df.sample(5)

,_id,title,reference
16605,66635665c2f544363eeb2d3c,Quyền và nghĩa vụ của tổ chức đấu giá tài sản ...,Tại Điều 24 Luật Đấu giá tài sản 2016 quyền và...
10015,66635715c2f544363eed9e59,Lao động nam có được hưởng chế độ thai sản khi...,Tại Điều 31 Luật Bảo hiểm xã hội 2014 có quy đ...
4307,66635725c2f544363eee1201,Có được chuyển đất phi nông nghiệp không thu t...,Căn cứ theo Điều 57 Luật Đất đai 2013 quy định...
11381,66635707c2f544363eed5361,Nghĩa vụ của tổ chức hành nghề luật sư gồm nhữ...,Căn cứ quy định Điều 40 Luật Luật sư 2006 quy ...
5359,66635714c2f544363eed8c66,Căn cứ để xác định mức độ hoàn thành công việc...,Căn cứ quy định khoản 1 Điều 36 Bộ luật Lao độ...


In [11]:
import re

pattern = r"(?:khoản (\d+)\s)?Điều (\d+)\s(.+?)\s(\d{4})"

def extract_law_info(reference):
    matches = re.findall(pattern, reference)
    article_indices = []
    clauses = []
    law_names = []
    years = []
    for match in matches:
        clause = match[0]
        article_index = match[1]
        law_name = match[2]
        year = match[3]

        if clause:
            clauses.append(int(clause))
        else:
            clauses.append(None)

        article_indices.append(int(article_index))
        law_names.append(law_name.strip())
        years.append(int(year))

    return article_indices, clauses, law_names, years

df[['article_index_arr', 'clause_arr', 'law_name_arr', 'year_arr']] = df['reference'].apply(lambda x: pd.Series(extract_law_info(str(x))))

df.sample(5)

,_id,title,reference,article_index_arr,clause_arr,law_name_arr,year_arr
407,66635754c2f544363eef15d7,3. Người cao tuổi được hưởng chính sách bảo tr...,Theo Điều 17 Luật người cao tuổi 2009 quy định...,[17],[None],[Luật người cao tuổi],[2009]
14743,66635676c2f544363eebb5e7,Như thế nào là hiếp dâm theo quy định của Điều...,Căn cứ tại khoản 1 Điều 141 Bộ luật Hình sự 20...,"[141, 1]","[1, 23]","[Bộ luật Hình sự, Luật Sửa đổi Bộ luật Hình sự]","[2015, 2017]"
5517,6663568bc2f544363eebeac0,Cơ quan nào có thẩm quyền cấp chứng chỉ hành n...,Căn cứ theo khoản 1 Điều 109 Luật Thú y 2015 q...,[109],[1],[Luật Thú y],[2015]
10300,66635777c2f544363eefe667,Làm giả chứng chỉ ngoại ngữ để được ra trường ...,Tại Khoản 126 Điều 1 Luật sửa đổi Bộ luật Hình...,[1],[None],[Luật sửa đổi Bộ luật Hình sự],[2017]
16452,66635705c2f544363eed3726,Quyền của bên mua bảo hiểm như thế nào?,Tại khoản 1 Điều 21 Luật Kinh doanh bảo hiểm 2...,[21],[1],[Luật Kinh doanh bảo hiểm],[2022]


In [12]:
def check_article_index(article_index):
    return len(article_index) != 0

df = df[df['article_index_arr'].apply(check_article_index)]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24073 entries, 0 to 24646
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   _id                24073 non-null  object
 1   title              24073 non-null  object
 2   reference          24073 non-null  object
 3   article_index_arr  24073 non-null  object
 4   clause_arr         24073 non-null  object
 5   law_name_arr       24073 non-null  object
 6   year_arr           24073 non-null  object
dtypes: object(7)
memory usage: 1.5+ MB


In [14]:
def process_record(row):
    if len(row['article_index_arr']) == len(row['law_name_arr']) == len(row['year_arr']):
        new_records = []
        for i in range(len(row['article_index_arr'])):
            new_record = {
                'article_index': row['article_index_arr'][i],
                'law_name': row['law_name_arr'][i],
                'year': row['year_arr'][i],
                'clause': row['clause_arr'][i] if row['clause_arr'][i] else ''
            }

            for col in row.index:
                if col not in ['article_index_arr', 'law_name_arr', 'year_arr', 'clause_arr']:
                    new_record[col] = row[col]
            new_records.append(new_record)
        return pd.DataFrame(new_records)
    elif len(row['article_index_arr']) > 1 and len(row['year_arr']) == 1:
        new_records = []
        for i in range(len(row['article_index_arr'])):
            new_record = {
                'article_index': row['article_index_arr'][i],
                'law_name': row['law_name_arr'][i],
                'clause': row['clause_arr'][i] if row['clause_arr'][i] else '',
                'year': row['year_arr'][0]
            }
            for col in row.index:
                if col not in ['article_index_arr', 'law_name_arr', 'year_arr', 'clause_arr']:
                    new_record[col] = row[col]
            new_records.append(new_record)
        return pd.DataFrame(new_records)
    else:
        return pd.DataFrame([row])

df_question = pd.concat(df.apply(process_record, axis=1).tolist(), ignore_index=True)

In [15]:
df_question['law_name_lower'] = df_question['law_name'].str.lower()
df_question = df_question[~((df_question['law_name_lower'].str.contains("sửa đổi") | df_question['law_name_lower'].str.contains("bổ sung")) & (df_question['clause'] == ""))]
df_question.drop(columns=['law_name_lower'], inplace=True)

In [16]:
num_duplicate_rows = df_question.duplicated(subset='_id').sum()
print("Số lượng hàng có giá trị trùng lặp trong cột '_id':", num_duplicate_rows)

Số lượng hàng có giá trị trùng lặp trong cột '_id': 2530


In [17]:
df_question.sample(5)

,article_index,law_name,year,clause,_id,title,reference
16256,21,"Luật Khám bệnh, chữa bệnh",2023,,6663568ac2f544363eebdf47,Những ngôn ngữ nào sẽ được sử dụng trong việc ...,"Tại Điều 21 Luật Khám bệnh, chữa bệnh 2023 có ..."
7293,30,Luật Đầu tư,2020,,66635705c2f544363eed2f88,Dự án đầu tư nào phải xin chấp thuận chủ trươn...,Căn cứ theo Điều 30 Luật Đầu tư 2020 quy định ...
13213,8,Luật Hôn nhân và gia đình,2014,,6663561bc2f544363eea9f0e,Người cùng giới có được kết hôn với nhau có vi...,Tại Điều 8 Luật Hôn nhân và gia đình 2014 có q...
25205,93,Luật Xây dựng,2014,,66635787c2f544363ef021d9,Điều kiện cấp giấy phép xây dựng đối với nhà ở...,Căn cứ quy định Điều 93 Luật Xây dựng 2014 đượ...
23617,2,Luật Sửa đổi Bộ luật Hình sự,2017,2,66635776c2f544363eefcebd,Tội vi phạm quy định về quản lý chất thải nguy...,Căn cứ tại Điều 236 Bộ luật Hình sự 2015 được ...


In [18]:
def getlaws(database, collection):
    client = MongoClient('mongodb://localhost:27017/')

    db = client[database]
    collection = db[collection]

    documents = collection.find({}, {'_id': 1, 'name': 1, 'identifier': 1})

    # client.close()
    return documents

In [19]:
laws = getlaws('lawlaboratory', 'laws')
codes = getlaws('lawlaboratory', 'codes')
constitutions = getlaws('lawlaboratory', 'constitution')

In [20]:
df_laws = pd.DataFrame(laws)
df_codes = pd.DataFrame(codes)
# df_constitutions = pd.DataFrame(constitutions)

df_laws_combined = pd.concat([df_laws, df_codes], ignore_index=True)

In [21]:
df_laws_combined.sample(5)

,_id,name,identifier
76,6635282f3558517825b88f0a,"Luật sửa đổi, bổ sung một số điều của Luật Thể...",26/2018/QH14
22,663528223558517825b88e9a,"Luật Thực hiện dân chủ ở cơ sở của Quốc hội, s...",10/2022/QH15
361,663529923558517825b89146,Luật Nghĩa vụ quân sự,6-LCT/HĐNN7
192,6635288f3558517825b88ff4,"Luật Giáo dục đại học của Quốc hội, số 08/2012...",08/2012/QH13
350,6635293d3558517825b89130,Luật Bầu cử Đại biểu Quốc hội số 56-L/CTN của ...,56-L/CTN


In [22]:
def clean_law_name(name):
    name = re.sub(r'số \d+/\d+/[A-Za-z\d]+', '', name)
    name = re.sub(r'của Quốc hội', '', name)
    name = name.rstrip(',')
    name = name.strip()
    return name

df_laws_combined['name'] = df_laws_combined['name'].apply(clean_law_name)

In [23]:
df_laws_combined['name'] = df_laws_combined['name'].apply(clean_law_name)
df_laws_combined.sample(5)

,_id,name,identifier
310,663529003558517825b890e0,Luật Thanh tra,22/2004/QH11
80,663528333558517825b88f12,Luật Quốc phòng,22/2018/QH14
165,6635287a3558517825b88fbe,"Luật sửa đổi, bổ sung một số điều của Luật Thi...",39/2013/QH13
217,663528a73558517825b89026,Luật An toàn thực phẩm,55/2010/QH12
39,663528263558517825b88ebe,"Luật sửa đổi, bổ sung một số điều của Luật Tổ ...",65/2020/QH14


In [24]:
def extract_year(identifier):
    year = identifier.split('/')[1]
    return year

df_laws_combined['year'] = df_laws_combined['identifier'].apply(extract_year)
df_laws_combined.sample(5)

,_id,name,identifier,year
199,663528943558517825b89002,Luật Lưu trữ,01/2011/QH13,2011
290,663528ea3558517825b890b8,Luật Nhà ở,56/2005/QH11,2005
285,663528e03558517825b890ae,Luật Đấu thầu,61/2005/QH11,2005
370,653a41cdcdee224751bca5c5,Bộ luật Hàng hải,40/2005/QH11,2005
122,663528563558517825b88f68,Luật Kiểm toán Nhà nước,81/2015/QH13,2015


In [25]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available! Using GPU for embedding.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU for embedding.")

CUDA is available! Using GPU for embedding.


In [26]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('keepitreal/vietnamese-sbert', device=device)

In [27]:
embeddings_laws_combined = model.encode(df_laws_combined['name'].tolist())
embeddings_df = model.encode(df_question['law_name'].tolist())

In [28]:
embeddings_laws_combined = np.asarray(embeddings_laws_combined)
embeddings_df = np.asarray(embeddings_df)

In [29]:
similarities = util.pytorch_cos_sim(embeddings_df, embeddings_laws_combined)

relevant_doc_index = similarities.argmax(dim=1)

In [30]:
relevant_law_id = df_laws_combined['identifier'].iloc[relevant_doc_index].tolist()

df_question['relevant_law_id'] = relevant_law_id

In [31]:
df_question.sample(5)

,article_index,law_name,year,clause,_id,title,reference,relevant_law_id
20453,47,Luật Quản lý thuế,2019,,666357a3c2f544363ef06b29,Thời hạn khai bổ sung hồ sơ khai thuế có sai s...,Theo Điều 47 Luật Quản lý thuế 2019 quy định v...,38/2019/QH14
17148,89,Bộ luật Hình sự,2015,,66635776c2f544363eefd9b2,"Pháp nhân thương mại phạm tội tàng trữ, vận ch...",Căn cứ quy định Điều 89 Bộ luật Hình sự 2015 q...,100/2015/QH13
13648,21,Luật Bảo hiểm xã hội,2014,,66635716c2f544363eedb223,Người lao động có thể thỏa thuận với người sử ...,Căn cứ theo Điều 21 Luật Bảo hiểm xã hội 2014 ...,58/2014/QH13
24683,17,Luật đấu giá tài sản,2016,,66635667c2f544363eeb414c,Xin cấp lại Chứng chỉ hành nghề đấu giá có lâu...,Tại Điều 17 Luật đấu giá tài sản 2016 quy định...,01/2016/QH14
18996,101,Luật Thương mại,2005,,66635765c2f544363eef762f,Thuế suất khi xuất hóa đơn quà tặng là vàng đã...,Căn cứ Điều 101 Luật Thương mại 2005 quy định ...,36/2005/QH11


In [32]:
df_question['extracted_year'] = df_question['relevant_law_id'].apply(extract_year)

df_question['extracted_year'] = pd.to_numeric(df_question['extracted_year'], errors='coerce')
df_question.dropna(subset=['extracted_year'], inplace=True)

df_question['extracted_year'] = df_question['extracted_year'].astype('int64')
df_question = df_question[df_question['extracted_year'] == df_question['year']]
df_question.drop(columns=['extracted_year'], inplace=True)

In [33]:
df_question.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20655 entries, 0 to 26919
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   article_index    20655 non-null  int64 
 1   law_name         20655 non-null  object
 2   year             20655 non-null  int64 
 3   clause           20655 non-null  object
 4   _id              20655 non-null  object
 5   title            20655 non-null  object
 6   reference        20655 non-null  object
 7   relevant_law_id  20655 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.4+ MB


In [34]:
df_question.reset_index(drop=True, inplace=True)
df_question.sample(5)

,article_index,law_name,year,clause,_id,title,reference,relevant_law_id
12761,17,Luật Cảnh vệ,2017,,66635835c2f544363ef2039a,Nữ có thể tham gia vào lực lượng cảnh vệ không?,Căn cứ theo Điều 17 Luật Cảnh vệ 2017 quy định...,13/2017/QH14
20385,25,"Luật Phòng, chống rửa tiền",2022,,66635608c2f544363eea7c05,Đối tượng nào phải báo cáo khi thực hiện các g...,"Tại Điều 25 Luật Phòng, chống rửa tiền 2022 có...",14/2022/QH15
4417,10,Luật Bảo vệ bí mật nhà nước,2018,1,6663583dc2f544363ef27e6a,Căn cứ vào đâu để xác định độ mật của bí mật n...,Theo khoản 1 Điều 10 Luật Bảo vệ bí mật nhà nư...,29/2018/QH14
3838,17,Luật Doanh nghiệp,2020,,66635703c2f544363eed184f,Công an có được thành lập doanh nghiệp không?,Căn cứ theo Điều 17 Luật Doanh nghiệp 2020 có ...,59/2020/QH14
3284,58,"Luật Cán bộ, công chức",2008,,66635841c2f544363ef2bd38,Có mấy mức xếp loại chất lượng công chức?,"Tại Điều 58 Luật Cán bộ, công chức 2008 được s...",22/2008/QH12


In [35]:
df_question['relevant_law_id'] = df_question['relevant_law_id'].str.lower()

In [36]:
import re
df_question['relevant_doc'] = df_question.apply(
    lambda row: f"{row['relevant_law_id']}_{row['article_index']}_{row['clause']}" if row['clause'] else f"{row['relevant_law_id']}_{row['article_index']}",
    axis=1
)

df_question_final = df_question[['_id', 'title']].copy()
df_question_final.columns = ['query_id', 'query']

relevant_docs = df_question.groupby('_id')['relevant_doc'].apply(list).reset_index()

df_question_final = df_question_final.merge(relevant_docs, left_on='query_id', right_on='_id', how='left').drop(columns=['_id'])

df_question_final.rename(columns={'relevant_doc': 'relevant_docs'}, inplace=True)

In [37]:
df_question_final.sample(5)

,query_id,query,relevant_docs
17893,66635779c2f544363eefff16,Tội giết người trong trạng thái tinh thần bị k...,[100/2015/qh13_70_1]
7647,66635705c2f544363eed3569,Khi nào có sự tồn tại của Ủy ban kiểm toán tro...,[59/2020/qh14_137]
9734,66635838c2f544363ef23205,Nguyên tắc phối hợp giữa Cảnh sát biển Việt Na...,[33/2018/qh14_23]
15065,66635719c2f544363eeddd3c,Thời gian đóng BHXH đã được tính hưởng BHXH mộ...,[58/2014/qh13_5]
10934,6663561cc2f544363eeaa3fc,Người nước ngoài được nhập Quốc tịch Việt Nam ...,[24/2008/qh12_19]


In [38]:
df_question_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20655 entries, 0 to 20654
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   query_id       20655 non-null  object
 1   query          20655 non-null  object
 2   relevant_docs  20655 non-null  object
dtypes: object(3)
memory usage: 484.2+ KB


In [39]:
num_records_with_multiple_relevant_docs = df_question_final[df_question_final['relevant_docs'].apply(len) > 1].shape[0]

print("Number records relevant_docs > 1:", num_records_with_multiple_relevant_docs)

Number records relevant_docs > 1: 4085


In [40]:
df_question_final.sample(5)

,query_id,query,relevant_docs
6997,66635787c2f544363ef0274c,Hồ sơ phê duyệt kế hoạch lựa chọn nhà thầu đối...,[22/2023/qh15_41]
11706,66635841c2f544363ef2b740,Ngạch công chức là gì? Ngạch công chức có mấy ...,[22/2008/qh12_7_4]
19639,66635719c2f544363eedda95,Điều kiện để được hưởng chế độ ốm đau là gì?,[58/2014/qh13_25]
12874,66635706c2f544363eed3ae1,Phí bảo hiểm trong hợp đồng bảo hiểm nhân thọ ...,[08/2022/qh15_4_28]
7948,6663583dc2f544363ef27b23,Khiếu nại tại tiếp dân thì bao lâu được giải q...,[42/2013/qh13_28]


In [41]:
def clean_string(s):
    if isinstance(s, list):
        return [clean_string(str(item)) for item in s]
    try:
        return s.encode('utf-8', errors='replace').decode('utf-8')
    except Exception as e:
        print(f"Error encoding string: {s}, error: {e}")
        return s

df_question_final['query_id'] = df_question_final['query_id'].astype(str)

df_question_final['relevant_docs'] = df_question_final['relevant_docs'].apply(
    lambda x: clean_string(x) if isinstance(x, list) else x
)

for column in df_question_final.select_dtypes(include=[object]).columns:
    df_question_final[column] = df_question_final[column].apply(clean_string)

df_question_final = df_question_final.drop_duplicates(subset=['query_id'])
# df_question_final.to_json('data/evaluate/query_set_evaluate.json', orient='records', lines=True, force_ascii=False)

In [42]:
exploded_relevant_docs = df_question_final.explode('relevant_docs')
exploded_relevant_docs = exploded_relevant_docs.dropna(subset=['relevant_docs'])

all_relevant_docs = exploded_relevant_docs['relevant_docs'].dropna().unique().tolist()

In [43]:
def extract_before_underscore(s):
    match = re.match(r'([^_]+)_.*', s)
    if match:
        return match.group(1)
    return s

extracted_docs = [extract_before_underscore(doc) for doc in all_relevant_docs]

In [44]:
def getAllLaws(database_name, collection_name):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database_name]
    collection = db[collection_name]

    # lowercase_extracted_docs = [doc.lower() for doc in extracted_docs]
    # collection.update_many({}, [{"$set": {"identifier": {"$toLower": "$identifier"}}}])
    #
    # query = {"identifier": {"$in": lowercase_extracted_docs}}
    result = collection.find()

    return list(result)

In [45]:
laws = getAllLaws("lawlaboratory", "laws")
codes = getAllLaws("lawlaboratory", "codes")
constitution = getAllLaws("lawlaboratory", "constitution")

In [46]:
def remove_id_from_data(data):
    return [{key: value for key, value in item.items() if key != '_id'} for item in data]

In [47]:
import jsonlines

def save_to_jsonl(data, file_path):
    with jsonlines.open(file_path, mode='w') as writer:
        for item in data:
            writer.write(item)

In [48]:
def find_empty_laws(all_laws):
    empty_laws = []
    for law in all_laws:
        if not law.get('parts') and not law.get('chapters') and not law.get('articles'):
            empty_laws.append(law)
    return empty_laws

empty_laws = find_empty_laws(laws + codes + constitution)

In [49]:
def delete_empty_laws(database_name, collection_name, empty_laws):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database_name]
    collection = db[collection_name]

    for law in empty_laws:
        identifier = law['identifier']
        collection.delete_one({'identifier': identifier})

delete_empty_laws("lawlaboratory", "laws", empty_laws)

In [51]:
all_laws = laws + codes + constitution
all_laws = remove_id_from_data(all_laws)
all_laws = [law for law in all_laws if law.get('parts') or law.get('chapters') or law.get('articles')]

# save_to_jsonl(all_laws, 'data/backup/laws_merged.jsonl')